In [13]:
import pandas as pd
import os

In [233]:
import spotipy
# headers = {
#         'Authorization': 'Bearer ',
#     }
token = "BQAzN34FXVMnvAnbcgwk539IeE1KSee5btXTSnlO5RGyJSP5ZTcWP4ud3hztgW4fTeryJ_m86K-CN2PhNL83kBpUpnAg6FNbW0AuTZGT3Rw31YCzfxiIP_xbLel7E5IHkF4DJmd_PnXmECWoI8qYgIaZqo3mcOWGssK5FkXHY4gw-NGpREtqJElJnm9oAOJLcsEySN4G3eFUTI2h4eoDCtbYxPMPZEkT1WxtAGldQpK9gjOfUzFTdPuWr6P77kyrAc5s7Q1cZEdyAvGVFznPKTXkeT_RhAQ"
spotify = spotipy.Spotify(auth=token)
results = spotify.category_playlists(category_id='mood')

In [235]:
def get_playlist_tracks(username,playlist_id):
    results = spotify.user_playlist_tracks(username,playlist_id)
    tracks = results['items']

    return tracks

In [244]:
id_ =[]
name=[]
play = []
url = []
for x in results['playlists']['items'][:10]:
    user = x['owner']['id']
    playlist = x['id']
   # print(user,playlist)
    for y in get_playlist_tracks(user,playlist):
        id_.append(y['track']['id'])
        name.append(y['track']['name'])
        url.append(y['track']['external_urls']['spotify'])
        play.append(x['id'])

In [245]:
df = pd.DataFrame(columns=['id', 'song_name', 'playlist_id','track_url'])

In [246]:
df['id'] = id_
df['song_name'] = name
df['playlist_id'] = play
df['track_url'] = url

In [247]:
df.song_name = [df.song_name[i].replace("'"," ") for i in range(len(df))]

In [248]:
df.head()

,id,song_name,playlist_id,track_url
0,1FHNctV68GUNLgXclG2DtR,Dancing in the Moonlight,37i9dQZF1DWSqmBTGDYngZ,https://open.spotify.com/track/1FHNctV68GUNLgX...
1,5nNmj1cLH3r4aA4XDJ2bgY,September,37i9dQZF1DWSqmBTGDYngZ,https://open.spotify.com/track/5nNmj1cLH3r4aA4...
2,5XzmZjXhMjDHr7ZfJ6DELQ,FourFiveSeconds,37i9dQZF1DWSqmBTGDYngZ,https://open.spotify.com/track/5XzmZjXhMjDHr7Z...
3,2tUBqZG2AbRi7Q0BIrVrEj,I Wanna Dance with Somebody (Who Loves Me),37i9dQZF1DWSqmBTGDYngZ,https://open.spotify.com/track/2tUBqZG2AbRi7Q0...
4,4o6BgsqLIBViaGVbx5rbRk,You Make My Dreams - Remastered,37i9dQZF1DWSqmBTGDYngZ,https://open.spotify.com/track/4o6BgsqLIBViaGV...


In [88]:
categories = []
for i in range(len(spotify.categories()['categories']['items'])):
    categories.append(spotify.categories()['categories']['items'][i]['id'])

In [135]:
playlist_ = []
playlist_id = []
results = spotify.category_playlists(category_id='mood')
for i in range(len(results['playlists']['items'])):
    user = results['playlists']['items'][i]['owner']['id']
    playlist = results['playlists']['items'][i]['id']

    playlist_.append(spotify.user_playlist(user, playlist)['name'])
    playlist_id.append(spotify.user_playlist(user, playlist)['id'])

In [136]:
playlist_df = pd.DataFrame(columns=['playlist_id', 'playlist_name'])
playlist_df['playlist_id'] = playlist_id
playlist_df['playlist_name'] = playlist_

In [207]:
playlist_df.head()

,playlist_id,playlist_name
0,37i9dQZF1DWSqmBTGDYngZ,Songs to Sing in the Shower
1,37i9dQZF1DWU0ScTcjJBdj,Relax & Unwind
2,37i9dQZF1DX7KNKjOK0o75,Have a Great Day!
3,37i9dQZF1DX7gIoKXt0gmx,All The Feels
4,37i9dQZF1DX3rxVfibe1L0,Mood Booster


In [138]:
len(playlist_df)

20

In [153]:
def doQuery( conn ) :
    cur = conn.cursor()
    for i in range(len(playlist_df)):
        cur.execute( "INSERT INTO playlist_spotify VALUES('"+playlist_df['playlist_name'][i]+"','"+playlist_df['playlist_id'][i]+"');")
        cur.execute("COMMIT;")

In [258]:
import psycopg2
myConnection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
doQuery( myConnection )
myConnection.close()

In [105]:
playlist_df['playlist_name'][0]

"Today's Top Hits"

In [255]:
def doQuery( conn ) :
    cur = conn.cursor()
    for i in range(len(df)):
        cur.execute( "INSERT INTO track_list VALUES('"+df['id'][i]+"','"+df['song_name'][i]+"','"+df['playlist_id'][i]+"','"+df['track_url'][i]+"');")
        cur.execute("COMMIT;")

In [253]:
def doQuery( conn ) :
    cur = conn.cursor()
    for i in range(len(df)):
        cur.execute( "SELECT * from track_list;")
        cur.execute("COMMIT;")